In [113]:
import keras 
from keras import applications
import matplotlib.pylab as plt
%matplotlib inline

import random
import time

In [114]:
# load trained vgg 16
pretrained_vgg_16 = applications.VGG16(weights= 'imagenet', include_top= True)

In [115]:
from keras.models import Model

# found with pretrained_vgg_16.layers[-2].get_config()
layer_name = 'fc2'

#get penultimate activation layer (relu activation layer)
intermediate_layer_model = Model(inputs = pretrained_vgg_16.input, outputs = pretrained_vgg_16.get_layer(layer_name).output)

In [ ]:
import os

#create list of all image paths to add the database
folder = "train_dog_breeds"

image_list = []

folder_list = os.listdir(folder)
for image_name in folder_list:
    image_path = os.path.join(folder, image_name)
    image_list.append(image_path)

In [ ]:
import cv2

feature_list = []
image_database = []

for image in image_list[:200]:
    #read image
    image = cv2.imread(image)
    #convert from BGR to RGB
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    #add image to image_list
    image_database.append(image)
    #resize for VGG
    image = cv2.resize(image, (224, 224))
    
    #add dummy dimenstion
    image = image[None]
    #extract feature vec for each image
    feature_vec = intermediate_layer_model.predict(image)
    #concatonate vector to one big list
    feature_list.append(feature_vec.ravel())

In [ ]:
#turn list into array and run nearest neighbors on database

import numpy as np
np.array(feature_list).shape

In [ ]:
from sklearn.neighbors import NearestNeighbors

#get 3 nearest neighbors (0 is the image)
neighbors = NearestNeighbors(n_neighbors=6)

#"train data" (organizes NN data to make querying faster)
neighbors.fit(feature_list)

In [ ]:
import pandas as pd
dog_labels = pd.read_csv('~/Code/dog-labels.csv')

In [ ]:
query_image_index = 1
distances, indicies = neighbors.kneighbors(feature_list[query_image_index][None])

In [ ]:
query_image_name = image_list[query_image_index]
nn_image_names = [image_list[image_ind] for image_ind in indicies.ravel()]

In [ ]:
plt.figure()
query_image = cv2.imread(query_image_name)
query_image = cv2.cvtColor(query_image,cv2.COLOR_BGR2RGB)
plt.imshow(query_image)

plt.figure(figsize=(50,50))
for nn_index in range(5):
    nn_image = cv2.imread(nn_image_names[nn_index+1])
    nn_image = cv2.cvtColor(nn_image,cv2.COLOR_BGR2RGB)
    plt.subplot(1,5,nn_index+1)
    plt.imshow(nn_image)

In [ ]:
import pickle
with open('dogs_features_dict.pickle','rb') as f:
    data = pickle.load(f, encoding= 'latin1')

In [ ]:
resnet_neigh = NearestNeighbors(n_neighbors=6)
resnet_feature_vec_list = list(data.values())
resnet_neigh.fit(resnet_feature_vec_list)

In [ ]:
print(resnet_feature_vec_list)

In [ ]:
query_image_index = 1
distances, indicies = resnet_neigh.kneighbors(resnet_feature_vec_list[query_image_index][None])

In [ ]:
resnet_image_list = list(data.keys())

In [ ]:
print(resnet_image_list)

In [ ]:
query_image_name = resnet_image_list[query_image_index]
nn_image_names = [resnet_image_list[image_ind] for image_ind in indicies.ravel()]

In [ ]:
plt.figure()
query_image = cv2.imread(os.path.join(folder,query_image_name))
query_image = cv2.cvtColor(query_image,cv2.COLOR_BGR2RGB)
plt.imshow(query_image)

plt.figure(figsize=(50,50))
for nn_index in range(3):
    nn_image = cv2.imread(os.path.join(folder,nn_image_names[nn_index+1]))
    nn_image = cv2.cvtColor(nn_image,cv2.COLOR_BGR2RGB)
    plt.subplot(1,5,nn_index+1)
    plt.imshow(nn_image)

In [ ]:
X = list(data.values())
image_name = list(data.keys())
import pandas as pd
labels_df = pd.read_csv('dog-labels.csv')

In [ ]:
image_name_trimmed = [im_name.split('.')[0] for im_name in image_name]

In [ ]:
Y_list = []
for im_name in image_name_trimmed:
    dog_breed_label = labels_df[labels_df['id'] == im_name]['breed'].values[0]
    Y_list.append(dog_breed_label)

In [ ]:
Y_list

In [ ]:
X = np.array(X)
X_train = X[:8000]
Y_train = Y_list[:8000]

X_val = X[8000:9000]
Y_val = Y_list[8000:9000]

X_test = X[9000:]
Y_test = Y_list[9000:]

In [ ]:
from sklearn.svm import LinearSVC

In [ ]:
my_svm = LinearSVC(C=.0001)
my_svm.fit(X_train,Y_train)

In [ ]:
my_svm.score(X_val,Y_val)

In [ ]:
val_pred = my_svm.predict(X_val)

In [ ]:
val_pred[:10]

In [ ]:
Y_val[:10]

In [ ]:
from keras.applications.resnet50 import ResNet50
from keras.preprocessing import image
from keras.applications.resnet50 import preprocess_input, decode_predictions
import numpy as np

model = ResNet50(weights='imagenet')

In [ ]:
#Example 1

In [ ]:
img_path = '/Users/Kyle/Code/sophiedog1.jpg'
img = image.load_img(img_path, target_size=(224, 224))
x = image.img_to_array(img)
x = np.expand_dims(x, axis=0)
x = preprocess_input(x)

preds = model.predict(x)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]


In [ ]:
img

In [105]:
#Example 2

In [ ]:
img_path2 = '/Users/Kyle/Code/coledog1.jpg'
img2 = image.load_img(img_path2, target_size=(224, 224))
x2 = image.img_to_array(img2)
x2 = np.expand_dims(x2, axis=0)
x2 = preprocess_input(x2)

preds = model.predict(x2)
# decode the results into a list of tuples (class, description, probability)
# (one such list for each sample in the batch)
print('Predicted:', decode_predictions(preds, top=3)[0])
# Predicted: [(u'n02504013', u'Indian_elephant', 0.82658225), (u'n01871265', u'tusker', 0.1122357), (u'n02504458', u'African_elephant', 0.061040461)]


In [ ]:
img2